In [1]:
from trainer import Trainer
from constrast_trainer import ContrastTrainer
from meta_featured_trainer import MetaFeatureTrainer
from resnet import fineTuneResNet50,lessFilterResNet50, paperResNet18, nnResNet, get_ensemble_resnet, ConstrastResnet
from senet import senetXX_generic
from swwae_trainer import SWWAETrainer
from swwae import get_swwae
import pandas as pd
import numpy as np
%matplotlib notebook

In [2]:
from sklearn.model_selection import KFold
fold_num = 2

In [3]:
import torch
from torch.optim import lr_scheduler
import gc

In [4]:
test_df = pd.read_json('Data/test.json', dtype={'inc_angle': str})
test_ids = test_df.id.tolist()

def is_real(angle):
    return len(str(angle).split('.')[1]) <= 4

pseudo_mask = test_df.inc_angle.apply(is_real)
print(pseudo_mask.value_counts())
#test_df = pd.read_json('Data/test.json')
print(test_df.shape)
test_df.head(5)

False    4999
True     3425
Name: inc_angle, dtype: int64
(8424, 4)


,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.9664
1,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",4023181e,32.6150720517
2,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",b20200e4,37.5054331594
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.4739
4,"[-23.37459373474121, -26.02718162536621, -28.1...","[-25.72234344482422, -27.011577606201172, -23....",4371c8c3,43.9188739832


In [5]:
test_ids = test_df['id']
print(test_ids)

0       5941774d
1       4023181e
2       b20200e4
3       e7f018bb
4       4371c8c3
5       a8d9b1fd
6       29e7727e
7       92a51ffb
8       c769ac97
9       aee0547d
10      565b28ac
11      e04e9775
12      8e8161d1
13      4cf4d256
14      139e5324
15      f156976f
16      68a117cc
17      d9aa7a56
18      9005b143
19      5f6d3988
20      9ad70954
21      b9087b9e
22      a39a1427
23      82fbe8ed
24      1fae4879
25      6dd8f13d
26      bbad5958
27      54527583
28      be8fa29c
29      81a3328f
          ...   
8394    8ae30ce6
8395    de27ed88
8396    66d5196f
8397    d85f1858
8398    16dcb33a
8399    eca3158e
8400    08daeee6
8401    e9c513ee
8402    b1519fa6
8403    dfc89540
8404    8fd8c0e9
8405    45df6347
8406    bf7928d7
8407    7b587c05
8408    c2834388
8409    146143c3
8410    d59aee00
8411    cbc0b93b
8412    088e2ff7
8413    673d33cd
8414    674b031e
8415    43db4207
8416    156855e1
8417    ac96cfb0
8418    fe45aef5
8419    16ee9b50
8420    5a599eb7
8421    df30d6

In [6]:
del test_df
gc.enable()
gc.collect()

20

In [7]:
test_df = pd.read_json('Data/no_denoise_processed_test.json')
#test_df.inc_angle = (test_df.inc_angle / 0.3).astype(np.int)

#test_df.reset_index(drop=True, inplace=True)
print(test_df.columns)
test_df.head(3)

Index(['band_mixed', 'inc_angle', 'is_iceberg'], dtype='object')


,band_mixed,inc_angle,is_iceberg
0,"[[[0.3602381271, 0.3185865893, 0.8586908419], ...",34.966400,0
1,"[[[0.23705839, 0.2707230281, 0.8586908419], [0...",32.615072,0
10,"[[[0.3381528741, 0.2572200913, 0.8586908419], ...",35.782900,0


In [8]:
test_df.sort_index(inplace=True)
test_df.head(5)

,band_mixed,inc_angle,is_iceberg
0,"[[[0.3602381271, 0.3185865893, 0.8586908419], ...",34.966400,0
1,"[[[0.23705839, 0.2707230281, 0.8586908419], [0...",32.615072,0
2,"[[[0.381044323, 0.39847112970000004, 0.8586908...",37.505433,0
3,"[[[0.4048886681, 0.28734042120000003, 0.858690...",34.473900,0
4,"[[[0.269489719, 0.2710927935, 0.8586908419], [...",43.918874,0


In [9]:
train_df = pd.read_json('Data/no_denoise_processed_train.json')
train_df.sort_index(inplace=True)
#train_df.inc_angle = pd.to_numeric(train_df.inc_angle, errors='coerce')
#print(train_df.shape)

In [10]:
print(train_df.head(3))

                                          band_mixed  inc_angle  is_iceberg
0  [[[0.2150773823, 0.2544778728, 0.8586908419], ...    43.9239           0
1  [[[0.4039837988, 0.20397305870000002, 0.858690...    38.1562           0
2  [[[0.2546405418, 0.2809726554, 0.8586908419], ...    45.2859           1


In [11]:
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = np.zeros((len(test_ids),))

top3_submission = pd.DataFrame()
top3_submission['id'] = test_ids
top3_submission['is_iceberg'] = np.zeros((len(test_ids),))

In [12]:
def predict_with_oof_models(seed=0):
    global fold_num
    global test_ids
    
    final_predictions = []
    final_top3_predictions = []
    
    for counter in range(fold_num):

        is_transfer_learning = False
        model = ConstrastResnet()
        #model = nnResNet(meta_features=1)  #dpn92(num_classes=1) #lessFilterResNet50()
        model_name= 'Trained_model/contrast_resnet_' + str(counter+2) + '.db'
        # model_name = 'Trained_model/nn_resnet_soft_pl_n_valid_' + str(counter+1)+ '.db'
        # testing snap shot for essemble
        optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

        tr = ContrastTrainer(
                model,
                optimizer,
                epochs=1800,
                milestones=[300, 1100, 1600],
                gamma=0.2,
                batch_size=128, 
                use_cuda=True, 
                gpu_idx=0,
                best_model_name = model_name,
                seed=seed,
                verbose=0)

        tr.load_checkpoint()
        
        data_augmentation_args = {
            'mirror': False, # not useful here
            'rotate': True,
            'scale': True,
            'translation': True
        }
        
        predictions, top_3_avg_predictions = tr.test(train_df, test_df[pseudo_mask].reset_index(drop=True), is_transfer_learning=is_transfer_learning, is_augment=False, data_augmentation_args=data_augmentation_args)
        predictions = np.array(predictions)
        top_3_avg_predictions = np.array(top_3_avg_predictions)
        
        predictions = predictions.reshape((predictions.shape[0],))
        top_3_avg_predictions = top_3_avg_predictions.reshape((predictions.shape[0],))
        
        final_predictions.append(predictions)
        final_top3_predictions.append(top_3_avg_predictions)
        
        del model, optimizer, tr
        
    return final_predictions, final_top3_predictions

In [13]:
repeat_count = 1
for i in range(repeat_count):
    print(i, 'round')
    final_predictions, final_top3_predictions = predict_with_oof_models(seed=i)
    for j, p in enumerate(final_predictions):
        submission.loc[pseudo_mask, 'is_iceberg_r' + str(i) + '_p' + str(j) ] = p
        top3_submission.loc[pseudo_mask, 'is_iceberg_r' + str(i) + '_p' + str(j) ] = final_top3_predictions[j]

0 round
epoch= 194 best_loss= 0.18154485523700714
epoch= 161 best_loss= 0.17424925665060678
gpu: 0  available: True
epoch= 59 best_loss= 0.2483526070912679
0 [0.0028731226921081543, 0.0029660463333129883, 0.0030241012573242188] [0.0053665992, 0.0061803097, 0.0067811813] 0.00411986093968 0.00453189341351
1 [0.97933263, 0.97805107, 0.97730654] [0.97619794309139252, 0.97619295865297318, 0.97595340386033058] 0.977765284479 0.977172423775
2 [0.0077142715454101563, 0.0078968405723571777, 0.0079088211059570313] [0.011666765, 0.012583503, 0.013322163] 0.00969051849097 0.010182060767
3 [0.98947841, 0.98838335, 0.98772997] [0.98922157101333141, 0.9891761876642704, 0.988916608504951] 0.989349990152 0.988817682645
4 [0.0028927922248840332, 0.0029871463775634766, 0.0030459165573120117] [0.0053137778, 0.0061079538, 0.0066913483] 0.00410328502767 0.00450648918437
5 [0.0013116002082824707, 0.0013625025749206543, 0.0014182329177856445] [0.0029122462, 0.0034904166, 0.003888451] 0.00211192318238 0.002397

58 [0.004562079906463623, 0.0046989917755126953, 0.0047518610954284668] [0.0074245906, 0.0083719455, 0.0090513155] 0.00599333527498 0.00647679739632
59 [0.98970759, 0.98856646, 0.98788595] [0.99019342940300703, 0.99014571495354176, 0.98987139575183392] 0.989950509276 0.989395089913
60 [0.94806802, 0.94619906, 0.94525993] [0.94595083966851234, 0.94590272009372711, 0.94560945406556129] 0.947009431198 0.946165004124
61 [0.0029644966125488281, 0.0030602216720581055, 0.0031188726425170898] [0.0054037464, 0.0062118452, 0.0068112952] 0.00418412149884 0.00459507961447
62 [0.0047472715377807617, 0.004889070987701416, 0.0049418210983276367] [0.0076894476, 0.0086588841, 0.009351423] 0.00621835957281 0.00671298638918
63 [0.00055462121963500977, 0.00058114528656005859, 0.00061833858489990234] [0.0015256969, 0.0019087555, 0.0021784906] 0.00104015908437 0.00122784136329
64 [0.0012107491493225098, 0.001259148120880127, 0.0013123154640197754] [0.0027853888, 0.0033480297, 0.0037328762] 0.00199806899764 

117 [0.0012608170509338379, 0.0013105273246765137, 0.0013647675514221191] [0.0028568737, 0.0034295174, 0.0038221625] 0.00205884536263 0.00234077758311
118 [0.0013796091079711914, 0.0014321804046630859, 0.001488029956817627] [0.0030896247, 0.0036917126, 0.0041085635] 0.00223461689893 0.00253162005295
119 [0.538252, 0.53691655, 0.53667724] [0.57262760400772095, 0.56927001476287842, 0.56905916333198547] 0.555439800024 0.553800428907
120 [0.0014622211456298828, 0.0015173554420471191, 0.0015749931335449219] [0.0031866736, 0.0037979954, 0.0042222771] 0.00232444738504 0.00262691930402
121 [0.99224621, 0.99133509, 0.99077421] [0.99243452865630388, 0.99238699721172452, 0.99214730225503445] 0.992340369616 0.991887391157
122 [0.0080041885375976563, 0.0081337094306945801, 0.0081911683082580566] [0.012677108, 0.013573918, 0.01424514] 0.0103406482376 0.0108042053568
123 [0.97922689, 0.97774589, 0.97689694] [0.97813539206981659, 0.9781044851988554, 0.97777783684432507] 0.978681139648 0.977981238936
1

176 [0.0022757649421691895, 0.0023534893989562988, 0.0024142265319824219] [0.0043659247, 0.0050795046, 0.0055937991] 0.00332084484398 0.00368045154028
177 [0.99114734, 0.99011236, 0.9894762] [0.99177185539156199, 0.99172018375247717, 0.99146561697125435] 0.991459597368 0.990948927278
178 [0.97933698, 0.97803932, 0.97729152] [0.97664067149162292, 0.97663228213787079, 0.97638346627354622] 0.977988824248 0.977387374267
179 [0.99271303, 0.99179357, 0.99121243] [0.99324927385896444, 0.99320019828155637, 0.99296838976442814] 0.992981154006 0.992522816096
180 [0.99391758, 0.99313366, 0.99264002] [0.99400183977559209, 0.99395471019670367, 0.99373591784387827] 0.993959712097 0.993563955805
181 [0.99186969, 0.99089593, 0.99029267] [0.99238955834880471, 0.99234014097601175, 0.99209774844348431] 0.992129623191 0.99164762185
182 [0.72260571, 0.72251827, 0.72235352] [0.72265440225601196, 0.72146955132484436, 0.72139871120452881] 0.722630053759 0.722166692217
183 [0.98950142, 0.98836321, 0.98767942] 

236 [0.0017358660697937012, 0.0017982721328735352, 0.0018571615219116211] [0.0036162797, 0.0042742006, 0.0047346568] 0.00267607287969 0.00300273947263
237 [0.98886836, 0.98769844, 0.98699945] [0.98901134729385376, 0.98896292317658663, 0.98868886940181255] 0.988939851522 0.98837156392
238 [0.99042714, 0.98935318, 0.98870903] [0.9907867694273591, 0.99073946103453636, 0.99048181716352701] 0.990606952924 0.990082899419
239 [0.021896600723266602, 0.021897971630096436, 0.022014021873474121] [0.027772598, 0.028873092, 0.029602502] 0.0248345993459 0.0253427975501
240 [0.99492717, 0.99417144, 0.99369252] [0.99540975037962198, 0.99536112742498517, 0.99516181601211429] 0.995168459136 0.994787303265
241 [0.8088842, 0.8079583, 0.80740851] [0.81832344830036163, 0.81825435161590576, 0.8176695853471756] 0.813603825867 0.813083067536
242 [0.060432314872741699, 0.062697410583496094, 0.062852382659912109] [0.067318469, 0.069120556, 0.069996238] 0.0638753920794 0.0654028952122
243 [0.0048600435256958008, 

...

3143 [0.054520070552825928, 0.054558634757995605, 0.054636538028717041] [0.030360306, 0.03317089, 0.034736458] 0.0424401881173 0.0436638162161
3144 [0.98520398, 0.98491412, 0.98457253] [0.98108887299895287, 0.98095986619591713, 0.98059592209756374] 0.983146427199 0.982889216083
3145 [0.013323962688446045, 0.013475775718688965, 0.013476133346557617] [0.0080074975, 0.0093957316, 0.009967342] 0.0106657301076 0.0112744071521
3146 [0.0038968324661254883, 0.0039663910865783691, 0.0039674639701843262] [0.0027973945, 0.0035871235, 0.0038501075] 0.00334711349569 0.00367755217788
3147 [0.98828667, 0.98781502, 0.98757088] [0.98419852368533611, 0.98405510745942593, 0.98376763984560966] 0.986242598854 0.985948974888
3148 [0.73693776, 0.7359423, 0.73588538] [0.81004828214645386, 0.80899706482887268, 0.80680346488952637] 0.773493021727 0.772435709834
3149 [0.0040698647499084473, 0.0041415095329284668, 0.004142463207244873] [0.002930972, 0.0037507161, 0.004024351] 0.00350041838828 0.00384331277261
315

3202 [0.95362252, 0.9533909, 0.9526121] [0.95730743184685707, 0.95728431642055511, 0.95629522204399109] 0.955464975908 0.955085414772
3203 [0.0075154900550842285, 0.0076187252998352051, 0.0076198577880859375] [0.0048238332, 0.0059730238, 0.0063769869] 0.00616966164671 0.00665465284449
3204 [0.034604430198669434, 0.034765362739562988, 0.034801483154296875] [0.019743925, 0.021998718, 0.023088053] 0.0271741775796 0.0281669953838
3205 [0.84649831, 0.84634769, 0.84615558] [0.88401664793491364, 0.88375493884086609, 0.88138209283351898] 0.86525747925 0.864692543944
3206 [0.027090013027191162, 0.027275800704956055, 0.027293741703033447] [0.0155989, 0.01756243, 0.018457081] 0.021344456356 0.0222129944401
3207 [0.0028839111328125, 0.0029412508010864258, 0.002942204475402832] [0.0021614588, 0.002811349, 0.0030240517] 0.00252268498298 0.00279403765065
3208 [0.17283695936203003, 0.1733439564704895, 0.17384237051010132] [0.092980526, 0.098120674, 0.10116509] 0.132908742875 0.135381595542
3209 [0.040

3261 [0.014343023300170898, 0.014503180980682373, 0.014503777027130127] [0.0086385002, 0.010090729, 0.010669108] 0.0114907617681 0.0121247197191
3262 [0.98879158, 0.98830402, 0.9880783] [0.98501660767942667, 0.98487753979861736, 0.98460640478879213] 0.986904096324 0.986612408732
3263 [0.013808250427246094, 0.013963043689727783, 0.013963818550109863] [0.0082721738, 0.0096842321, 0.010265569] 0.0110402121209 0.0116595146246
3264 [0.017411351203918457, 0.017579555511474609, 0.017582356929779053] [0.010266199, 0.011829269, 0.012479863] 0.0138387749903 0.0145247657783
3265 [0.02235943078994751, 0.022532463073730469, 0.02254486083984375] [0.013650599, 0.015407993, 0.016198713] 0.0180050148629 0.0187823432498
3266 [0.019606471061706543, 0.019780218601226807, 0.019787013530731201] [0.012666601, 0.014320961, 0.015061327] 0.016136535909 0.0168704320677
3267 [0.010120511054992676, 0.010246992111206055, 0.010247588157653809] [0.0062230583, 0.0075408262, 0.0080258045] 0.00817178469151 0.00873413006

3319 [0.99263728, 0.99230206, 0.9921577] [0.98852111119776964, 0.98837820161134005, 0.98819693922996521] 0.990579193924 0.990365547749
3320 [0.62742871, 0.62623477, 0.62598789] [0.72296220064163208, 0.72145119309425354, 0.71940430998802185] 0.675195455551 0.673911511898
3321 [0.99451739, 0.99421614, 0.99412173] [0.99059527646750212, 0.9904500674456358, 0.99031565338373184] 0.992556331214 0.992369376278
3322 [0.065793752670288086, 0.065885365009307861, 0.065937697887420654] [0.036037393, 0.039063394, 0.040895119] 0.0509155727923 0.0522687869767
3323 [0.98467815, 0.98430139, 0.98396599] [0.98147203214466572, 0.98134233243763447, 0.98097795993089676] 0.983075090684 0.982789642488
3324 [0.99664646, 0.99636286, 0.99633968] [0.99277610564604402, 0.99262647936120629, 0.9925515353679657] 0.994711284759 0.99455052133
3325 [0.79144704, 0.79005259, 0.78973979] [0.85781629383563995, 0.85739235579967499, 0.85466781258583069] 0.824631668627 0.823519314329
3326 [0.98777592, 0.98728812, 0.98703718] [0

3378 [0.0048084259033203125, 0.0048870444297790527, 0.0048877596855163574] [0.0032957054, 0.0041893357, 0.0044865971] 0.00405206566211 0.00442581138729
3379 [0.98729825, 0.9868421, 0.9865759] [0.98371322266757488, 0.98357441835105419, 0.98327111639082432] 0.985505736433 0.985212500828
3380 [0.009244382381439209, 0.0093632340431213379, 0.0093640685081481934] [0.0057789013, 0.0070693623, 0.0075302483] 0.00751164183021 0.0080583661329
3381 [0.99098456, 0.99057591, 0.99038869] [0.98709960468113422, 0.98695439845323563, 0.98673918936401606] 0.98904208187 0.9887903921
3382 [0.99320328, 0.99287462, 0.99274629] [0.98935013264417648, 0.98921297583729029, 0.98904792498797178] 0.9912767075 0.991072538619
3383 [0.98082721, 0.98057449, 0.98006541] [0.97830961458384991, 0.97820186428725719, 0.97774784080684185] 0.979568413459 0.97928773767
3384 [0.096725642681121826, 0.096964538097381592, 0.097015202045440674] [0.051573224, 0.055447716, 0.0579326] 0.0741494335234 0.0759431539724
3385 [0.9731856, 0.9

In [14]:
print(train_df.loc[0, 'is_iceberg'])

0


In [15]:
submission['is_iceberg_max'] = submission.iloc[:, 2:repeat_count*fold_num+2].max(axis=1)
submission['is_iceberg_min'] = submission.iloc[:, 2:repeat_count*fold_num+2].min(axis=1)
submission['is_iceberg_mean'] = submission.iloc[:, 2:repeat_count*fold_num+2].mean(axis=1)
submission['is_iceberg_median'] = submission.iloc[:, 2:repeat_count*fold_num+2].median(axis=1)

top3_submission['is_iceberg_max'] = top3_submission.iloc[:, 2:repeat_count*fold_num+2].max(axis=1)
top3_submission['is_iceberg_min'] = top3_submission.iloc[:, 2:repeat_count*fold_num+2].min(axis=1)
top3_submission['is_iceberg_mean'] = top3_submission.iloc[:, 2:repeat_count*fold_num+2].mean(axis=1)
top3_submission['is_iceberg_median'] = top3_submission.iloc[:, 2:repeat_count*fold_num+2].median(axis=1)

In [16]:
submission['is_iceberg'] = submission['is_iceberg_mean']
submission[['id', 'is_iceberg']].to_csv('Submissions/submission_ensemble_contrast_resnet_oof_augment_avg.csv', 
                                        index=False, float_format='%.15f')

top3_submission['is_iceberg'] = top3_submission['is_iceberg_mean']
top3_submission[['id', 'is_iceberg']].to_csv('Submissions/submission_ensemble_contrast_resnet_top3_oof_augment_avg.csv', 
                                        index=False, float_format='%.15f')

In [17]:
cutoff_hi=.1
cutoff_lo=.9
submission['is_iceberg'] = np.where(np.all(submission.iloc[:,2:repeat_count*fold_num+2] > cutoff_lo, axis=1), 
                                    submission['is_iceberg_max'], # all > lo
                                    np.where(np.all(submission.iloc[:,2:repeat_count*fold_num+2] < cutoff_hi, axis=1),
                                             submission['is_iceberg_min'], # all < hi 
                                             submission['is_iceberg_median'] # use the best
                                            )
                                   )

top3_submission['is_iceberg'] = np.where(np.all(top3_submission.iloc[:,2:repeat_count*fold_num+2] > cutoff_lo, axis=1), 
                                    top3_submission['is_iceberg_max'], # all > lo
                                    np.where(np.all(top3_submission.iloc[:,2:repeat_count*fold_num+2] < cutoff_hi, axis=1),
                                             top3_submission['is_iceberg_min'], # all < hi 
                                             top3_submission['is_iceberg_median'] # use the best
                                            )
                                   )

In [18]:
submission[['id', 'is_iceberg']].to_csv('Submissions/submission_ensemble_contrast_resnet_oof_augment_minmax_median.csv'.format(repeat_count)
                  , float_format="%.15f", index=False) 

top3_submission[['id', 'is_iceberg']].to_csv('Submissions/submission_ensemble_contrast_resnet_top3_oof_augment_minmax_median.csv'.format(repeat_count)
                  , float_format="%.15f", index=False) 

In [2]:
import pandas as pd
test = pd.read_csv('Submissions/submission_ensemble_contrast_resnet_top3_oof_augment_avg.csv')
test['is_iceberg'] = pd.to_numeric(test['is_iceberg'], errors='coerce')
test['is_iceberg'].fillna(value=0, inplace=True)
print(test.head(3))

         id  is_iceberg
0  5941774d     0.00699
1  4023181e     0.00000
2  b20200e4     0.00000


In [3]:
test.to_csv('Submissions/submission_ensemble_contrast_resnet_top3_oof_augment_avg.csv', index=False, float_format='%.15f')